In [2]:


import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np

In [1]:
!pip install --upgrade --force-reinstall gensim --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 140.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 140.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 159.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: smart-open
    Found existing installation: smart-open 7.1.0
    Uninstalling smart-open-7.1.0:
      Successful

In [3]:
# Load the dataset
dataset = pd.read_csv('/content/reddit_preprocessing.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

In [4]:
# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

In [5]:
# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

In [6]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=300, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

In [7]:
# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

In [8]:
# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

In [9]:
X_train_word2vec.shape

(29329, 300)

In [10]:
# Define the LightGBM model with your best settings
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=0.1,  # L2 regularization,
    learning_rate=0.08081298097796712,
    n_estimators=367,
    max_depth=20
)

In [11]:
# Train the LightGBM model on Word2Vec embeddings
best_model.fit(X_train_word2vec, y_train_cleaned)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.241094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 300
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


LGBMClassifier(class_weight='balanced', is_unbalance=True,
               learning_rate=0.08081298097796712, max_depth=20,
               metric='multi_logloss', n_estimators=367, num_class=3,
               objective='multiclass', reg_alpha=0.1, reg_lambda=0.1)

In [12]:
# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [13]:
print(classification_report(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

          -1       0.48      0.49      0.49      1647
           0       0.73      0.73      0.73      2510
           1       0.69      0.69      0.69      3176

    accuracy                           0.66      7333
   macro avg       0.64      0.64      0.64      7333
weighted avg       0.66      0.66      0.66      7333



In [15]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 12.6 MB/s eta 0:00:00


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score

# Load the dataset
dataset = pd.read_csv('/content/reddit_preprocessing.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

# Encode the target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_cleaned)
y_test_encoded = label_encoder.transform(y_test_cleaned)

# Objective function for Optuna hyperparameter tuning
def objective(trial):
    # Suggest hyperparameters to be tuned
    params = {
        "objective": "multiclass",
        "num_class": 3,
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
        "reg_alpha": 0.1,  # L1 regularization
        "reg_lambda": 0.1,  # L2 regularization
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 20)
    }

    # Initialize the LightGBM model with suggested parameters
    model = LGBMClassifier(**params)

    # Perform cross-validation to evaluate the model performance
    scores = cross_val_score(model, X_train_word2vec, y_train_encoded, cv=3, scoring='accuracy')

    # Return the mean accuracy score across folds
    return scores.mean()

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Run 30 trials

# Print the best trial
print("Best trial:")
best_params = study.best_trial.params
print(best_params)

# Train the LightGBM model with the best parameters
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,
    reg_lambda=0.1,
    **best_params
)

best_model.fit(X_train_word2vec, y_train_encoded)

# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)




[I 2025-06-04 06:57:57,503] A new study created in memory with name: no-name-3b53caf3-0dc7-44c3-abf2-2ea492c1784e
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026928 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024411 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026596 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-06-04 08:36:27,861] Trial 29 finished with value: 0.6508575564047155 and parameters: {'n_estimators': 885, 'learning_rate': 0.233152894093664, 'max_depth': 9}. Best is trial 14 with value: 0.6518122475446874.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Best trial:
{'n_estimators': 873, 'learning_rate': 0.15848229095064684, 'max_depth': 10}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [17]:
# Print classification report
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.40      0.45      1647
           1       0.73      0.72      0.72      2510
           2       0.65      0.73      0.69      3176

    accuracy                           0.65      7333
   macro avg       0.63      0.62      0.62      7333
weighted avg       0.65      0.65      0.65      7333



In [18]:
!pip install optuna